In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import math
import json
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.amp.autocast_mode import autocast
from torch.amp.grad_scaler import GradScaler
from transformers import AutoTokenizer, AutoModel, get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score


In [ ]:
# DOWNLOAD PRETRAINED MODEL AND TOKENIZER
roberta_tokenizer = AutoTokenizer.from_pretrained("alex-shvets/roberta-large-emopillars-contextless")
roberta_base = AutoModel.from_pretrained("alex-shvets/roberta-large-emopillars-contextless")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device: ", device)

In [ ]:
# Remove the pooler layer
roberta_base.pooler = None
roberta_base.gradient_checkpointing_enable()

In [ ]:
HIDDEN_SIZE = roberta_base.config.hidden_size
print("Hidden size: ", HIDDEN_SIZE)

In [ ]:
train_path = r"C:\Users\arnab\Desktop\emopillar_train.csv"
val_path = r"C:\Users\arnab\Desktop\emopillar_validation.csv"
test_path = r"C:\Users\arnab\Desktop\emopillar_test.csv"

df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

In [ ]:
class EmoPillars_Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = 128
        self.target_cols = [str(i) for i in range(28)]
        self.soft_target_cols = [str(f"{i}_exp") for i in range(28)]

    def __len__(self):
        return(len(self.data))

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        text = str(item.utterance)
        encoding = self.tokenizer.encode_plus(text,
                                            add_special_tokens=True,
                                            truncation=True,
                                            return_tensors='pt',
                                            max_length=self.max_len,
                                            padding='max_length',
                                            return_attention_mask=True)

        target = torch.tensor(item[self.target_cols].values.astype('float32'))
        soft_target = torch.tensor(item[self.soft_target_cols].values.astype('float32'))

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "atten_masks": encoding["attention_mask"].squeeze(0),
            "hard_target": target,
            "soft_target": soft_target
        }

In [ ]:
data = EmoPillars_Dataset(df_train, roberta_tokenizer)

In [ ]:
len(data)

In [ ]:
data.__getitem__(1)

In [ ]:
# Data Loaders
train_dataloader = DataLoader(EmoPillars_Dataset(df_train, roberta_tokenizer), batch_size=64, num_workers=4, shuffle=True)
val_dataloader = DataLoader(EmoPillars_Dataset(df_val, roberta_tokenizer), batch_size=64, num_workers=4)
test_dataloader = DataLoader(EmoPillars_Dataset(df_test, roberta_tokenizer), batch_size=64, num_workers=4)

In [ ]:
with open(r"C:\Users\arnab\Desktop\label_embeddings.json", "r") as f:
    label_embeddings = json.load(f)

emo_emb=[]
for k, v in label_embeddings.items():
    emb = torch.tensor(v, dtype=torch.float32)
    emo_emb.append(emb)

emotion_bank = torch.cat(emo_emb, dim=0).to(device)
print(emotion_bank.shape)

In [ ]:
class Encoder(nn.Module):
    """
    Text Encoder:
    - Takes in tokenized text (from tokenizer)
    - Generates the text embedding vector
    """
    def __init__(self, base_encoder):
        super().__init__()
        self.encoder = base_encoder

    def forward(self, inputs):
        """
        inputs: tokenizer output dict (input_ids, attention_mask)
        """
        outputs = self.encoder(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]                                                                   # [B, T, H]

        atten_mask = inputs['attention_mask']                                                                           # [B, T]
        # Mean pooling text_embeddings
        atten_mask = atten_mask.unsqueeze(-1).float()
        pooled_text_emb = (last_hidden_state * atten_mask).sum(dim=1) / atten_mask.sum(dim=1).clamp(min=1e-9)           # [B, H]

        return {
            "text_embed": pooled_text_emb,
            "last_hidden_state": last_hidden_state,
            "atten_mask": atten_mask
        }

In [ ]:
class CrossAttentionModule(nn.Module):
    def __init__(self, emotion_bank, hidden_size=HIDDEN_SIZE):
        super().__init__()
        self.hidden_size = hidden_size
        self.emo_bank = emotion_bank
        self.register_buffer("emotion_bank", emotion_bank)

        # Multi-head cross attention
        self.cross_attn = nn.MultiheadAttention(
            embed_dim=self.hidden_size,
            num_heads=16,
            batch_first=True
        )
        self.rms_norm = nn.RMSNorm(hidden_size)

    def forward(self, encoder_out):
        B = encoder_out["last_hidden_state"].size(0)
        emo_bank = self.emo_bank.unsqueeze(0).expand(B, -1, -1)

        # cross-attention (query=text, key/value=emotion)
        attn_out, _ = self.cross_attn(
            query = encoder_out["last_hidden_state"],                        # [B, T, HIDDEN_SIZE]
            key = emo_bank,                                                  # [B, 28, HIDDEN_SIZE] (Key/Value)
            value = emo_bank,                                                # [B, 28, HIDDEN_SIZE]
        )

        # Fuse and pool
        fused_hidden_state = encoder_out["last_hidden_state"] + attn_out
        atten_mask = encoder_out["atten_mask"]
        fused_emo_text_emb = (fused_hidden_state * atten_mask).sum(dim=1) / atten_mask.sum(dim=1).clamp(min=1e-9)
        fused_emo_text_emb = self.rms_norm(fused_emo_text_emb)

        return fused_emo_text_emb

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim=HIDDEN_SIZE, num_classes=28):
        super().__init__()
        self.input_dim = input_dim

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.RMSNorm(512),
            nn.Linear(512, num_classes)
        )

    def forward(self, h):
        return self.mlp(h)

In [ ]:
# Main Model class
class EmoAxis(nn.Module):
    def __init__(self, encoder, cross_atten_module, classifier):
        super().__init__()
        self.encoder = encoder
        self.cross_atten = cross_atten_module
        self.classifier = classifier


    def total_params(self):
        """Utility function to check trainable vs total params."""
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Total parameters: {total:,}")
        print(f"Trainable parameters: {trainable:,}")


    def forward(self, input_ids, atten_mask):
        # Encoder
        outputs = self.encoder(
            inputs = {"input_ids": input_ids, "attention_mask": atten_mask}
            )
        # Cross Attention
        fused_emb = self.cross_atten(outputs)
        # Classifier
        logits = self.classifier(fused_emb)

        return logits


    def infer(self, input_ids, atten_mask):
        """
        Predict emotions from raw text (inference using prior p(z|x)).
        """
        self.eval()
        with torch.no_grad():
            logits = self.forward(input_ids, atten_mask)
            return logits

In [ ]:
encoder = Encoder(base_encoder=roberta_base)

cross_atten_module = CrossAttentionModule(emotion_bank=emotion_bank, hidden_size=HIDDEN_SIZE)
classifier = Classifier(input_dim=HIDDEN_SIZE)

# Initialize model
model = EmoAxis(
    encoder=encoder,
    cross_atten_module=cross_atten_module,
    classifier=classifier
)

model.total_params()

In [ ]:
def freeze_encoder_layers(encoder, freeze_upto: int=0):
    roberta_base_model = encoder.encoder

    for name, param in roberta_base_model.named_parameters():
        param.requires_grad = True

    if freeze_upto >= 0:
        for layer_idx in range(freeze_upto + 1):
            for param in roberta_base_model.encoder.layer[layer_idx].parameters():
                param.requires_grad = False

    print(f"\n\nFrozen encoder layers - 0 to {freeze_upto}\n\n")

In [ ]:
def evaluate(model, val_dataloader, device, threshold=0.5):
    """
    Evaluate the model on validation set.
    Returns:
        - avg_val_loss: per-sample BCE loss
        - micro_f1, macro_f1: multi-label F1 scores
    """
    model.eval()
    loss_func = nn.BCEWithLogitsLoss()
    val_loss = 0.0
    total_samples = 0

    preds_all = []
    truths_all = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['atten_masks'].to(device)
            hard_target = batch['hard_target'].to(device)

            # Forward pass
            logits = model.infer(input_ids, attention_mask)
            loss = loss_func(logits, hard_target)

            batch_size = input_ids.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            # Predictions
            probs = torch.sigmoid(logits)
            preds = (probs >= threshold).int()

            preds_all.append(preds.cpu())
            truths_all.append(hard_target.cpu().int())

    # Concatenate all batches
    preds_all = torch.cat(preds_all, dim=0).numpy()
    truths_all = torch.cat(truths_all, dim=0).numpy()

    # Compute metrics
    avg_val_loss = val_loss / total_samples
    micro_f1 = f1_score(truths_all, preds_all, average='micro', zero_division=0)
    macro_f1 = f1_score(truths_all, preds_all, average='macro', zero_division=0)

    return {
        "avg_val_loss": avg_val_loss,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

In [ ]:
def train(
    model: torch.nn.Module,
    train_dataloader,
    val_dataloader,
    device: torch.device,
    epochs: int = 12,
    lr_encoder: float = 2.5e-6,
    lr_other: float = 2e-5,
    weight_decay: float = 0.001,
    warmup_ratio: float = 0.07,
    gradient_accumulation_steps: int = 4,
    max_grad_norm: float = 1.0,
    use_amp: bool = True,
    early_stopping_patience: int = 3,
    min_epochs_before_stop: int = 3,
    use_grad_checkpointing: bool = True   # Optional memory optimization
):
    model.to(device)

    # --- Optional: enable gradient checkpointing ---
    if use_grad_checkpointing and hasattr(model.encoder.encoder, "gradient_checkpointing_enable"):
        print("✅ Gradient checkpointing enabled for encoder.")
        model.encoder.encoder.gradient_checkpointing_enable()

    # --- Scheduler setup ---
    steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    total_steps = steps_per_epoch * epochs
    warmup_steps = int(total_steps * warmup_ratio)

    loss_func = nn.BCEWithLogitsLoss()

    global_step = 0
    best_val_microF1 = -1.0
    epochs_no_improve = 0
    current_freeze_config = None

    history = {"train_loss": [], "val_loss": [], "micro_f1": [], "macro_f1": []}

    optimizer = None
    scheduler = None
    scaler = GradScaler(enabled=(use_amp and device.type == 'cuda'))   # moved scaler outside loop
    # --- Training Loop ---
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        total_samples = 0
        torch.cuda.empty_cache()
        gc.collect()

        # Progressive unfreezing for 24-layer RoBERTa
        # Stage 1: train only classifier (0-23 frozen)
        # Stage 2: unfreeze top 6 layers
        # Stage 3: unfreeze 12 layers
        # Stage 4+: fully unfreeze
        if epoch < 3:
            freeze_level = 23
        elif epoch < 6:
            freeze_level = 17
        elif epoch < 9:
            freeze_level = 11
        else:
            freeze_level = -1

        # Reset optimizer/scheduler only if freeze config changed
        if freeze_level != current_freeze_config:
            freeze_encoder_layers(model.encoder, freeze_upto=freeze_level)
            current_freeze_config = freeze_level

            encoder_params, other_params = [], []
            for name, p in model.named_parameters():
                if not p.requires_grad:
                    continue
                if 'encoder' in name:
                    encoder_params.append(p)
                else:
                    other_params.append(p)

            optimizer = AdamW([
                {"params": encoder_params, "lr": lr_encoder},
                {"params": other_params, "lr": lr_other}
            ], weight_decay=weight_decay)

            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=warmup_steps,
                num_training_steps=total_steps
            )

        # scaler = GradScaler(enabled=(use_amp and device.type == 'cuda'))
        optimizer.zero_grad(set_to_none=True)

        # --- Batch Loop ---
        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}/{epochs}")
        for step, batch in progress_bar:
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['atten_masks'].to(device, non_blocking=True)
            hard_target = batch['hard_target'].to(device, non_blocking=True)

            batch_size = input_ids.size(0)

            with autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                logits = model(input_ids=input_ids, atten_mask=attention_mask)
                loss = loss_func(logits, hard_target)
                total_loss = loss / gradient_accumulation_steps

            scaler.scale(total_loss).backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                global_step += 1

            epoch_loss += loss.item() * batch_size
            total_samples += batch_size

            # 🧹 Periodic GPU cleanup
            if step % 100 == 0 and step != 0:
                torch.cuda.empty_cache()
                gc.collect()

        avg_train_loss = epoch_loss / total_samples
        history["train_loss"].append(avg_train_loss)
        print(f"\n| Epoch {epoch+1}/{epochs} | Avg Train Loss: {avg_train_loss:.4f}")

        # --- Validation ---
        val_metrics = evaluate(model, val_dataloader, device)
        avg_val_loss = val_metrics.get("avg_val_loss", None)
        micro_F1 = val_metrics.get("micro_f1", -1)
        macro_F1 = val_metrics.get("macro_f1", -1)

        history["val_loss"].append(avg_val_loss)
        history["micro_f1"].append(micro_F1)
        history["macro_f1"].append(macro_F1)

        print(f"Validation | Avg Loss: {avg_val_loss:.4f}; Micro-F1: {micro_F1:.4f}; Macro-F1: {macro_F1:.4f}")

        # --- Early Stopping ---
        if micro_F1 > best_val_microF1:
            best_val_microF1 = micro_F1
            epochs_no_improve = 0
            os.makedirs("model", exist_ok=True)
            torch.save(model.state_dict(), os.path.join("model", "best_model.pt"))
            print("✅ Micro-F1 improved — model saved.")
        else:
            epochs_no_improve += 1
            print(f"⚠️ No improvement for {epochs_no_improve} epoch(s).")

        if epoch + 1 >= min_epochs_before_stop and epochs_no_improve >= early_stopping_patience:
            print("\n⛔ Early stopping activated.")
            break

        # Cleanup after each epoch
        torch.cuda.empty_cache()
        gc.collect()

    print(f"\n\n🎯 Training completed. Best validation micro-F1 = {best_val_microF1:.4f}")
    return model, history


In [ ]:
model, history = train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device
)
